## **LIMPIEZA Y TRANSFORMACIÓN DE LOS DATOS**

Este notebook realiza el proceso completo de limpieza y preparación de datos de recursos humanos.

**Pasos principales:**
1. Importación de librerías necesarias
2. Carga de datos originales
3. Normalización de nombres de columnas
4. Limpieza de duplicados y columnas sin valor
5. Normalización de datos categóricos
6. Conversión de tipos de datos
7. Mapeo de variables ordinales
8. Imputación de valores nulos
9. Exportación de datos limpios

In [31]:
# ============================================================================
# INSTALACIÓN E IMPORTACIÓN DE LIBRERÍAS
# ============================================================================

# Librerías para manipulación de datos
import pandas as pd  # Trabajo con DataFrames y análisis de datos
import numpy as np   # Operaciones numéricas y arrays
import re            # Expresiones regulares para procesamiento de texto

# Librerías de scikit-learn para imputación y escalado
from sklearn.impute import SimpleImputer, KNNImputer  # Métodos de imputación de nulos
from sklearn.preprocessing import StandardScaler      # Estandarización de variables numéricas

# Librería del sistema operativo
import os  # Gestión de rutas y archivos del sistema

In [32]:
# ============================================================================
# CONFIGURACIÓN DE VISUALIZACIÓN DE PANDAS
# ============================================================================

# Mostrar todas las columnas al visualizar DataFrames (sin truncar)
pd.set_option("display.max_columns", None)

# Mostrar hasta 100 filas (por defecto son 60)
pd.set_option("display.max_rows", 100)

In [33]:
# ============================================================================
# CARGA DE DATOS ORIGINALES
# ============================================================================

# Leer el archivo CSV con los datos de recursos humanos desde la carpeta raw
df = pd.read_csv('../data/raw/hr.csv')

# Mostrar las primeras 3 filas para verificar la carga correcta
df.head(3)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41.0,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,sALES eXECUTIVE,4.0,Single,5993.0,19479,8,Y,Yes,11,3,1,80.0,0,8,0.0,1,6,4,0,5.0
1,49.0,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,rESEARCH sCIENTIST,2.0,Married,5130.0,24907,1,Y,No,23,4,4,NaN,1,10,3.0,3,10,7,1,7.0
2,37.0,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,lABORATORY tECHNICIAN,3.0,Single,2090.0,2396,6,Y,Yes,15,3,2,NaN,0,7,3.0,3,0,0,0,0.0


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474 entries, 0 to 1473
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1401 non-null   float64
 1   Attrition                 1474 non-null   object 
 2   BusinessTravel            1357 non-null   object 
 3   DailyRate                 1474 non-null   int64  
 4   Department                1445 non-null   object 
 5   DistanceFromHome          1474 non-null   int64  
 6   Education                 1474 non-null   int64  
 7   EducationField            1416 non-null   object 
 8   EmployeeCount             1474 non-null   int64  
 9   EmployeeNumber            1474 non-null   int64  
 10  EnvironmentSatisfaction   1474 non-null   int64  
 11  Gender                    1474 non-null   object 
 12  HourlyRate                1474 non-null   int64  
 13  JobInvolvement            1474 non-null   int64  
 14  JobLevel

---
## **1. NORMALIZACIÓN DE NOMBRES DE COLUMNAS**

Transformamos los nombres de las columnas a un formato estándar (snake_case) para:
- Facilitar el acceso a las columnas sin errores de mayúsculas
- Mejorar la legibilidad del código
- Seguir las mejores prácticas de nomenclatura en Python

In [35]:
def normalizar_nombres_columnas(lista_columnas, mostrar_resumen=True):
    """
    Normaliza los nombres de las columnas de un DataFrame a formato snake_case.
    
    El proceso de normalización incluye:
    - Eliminar espacios al inicio y al final
    - Eliminar caracteres especiales (excepto guiones bajos)
    - Convertir de CamelCase o PascalCase a snake_case
    - Convertir todo a minúsculas
    
    Parámetros:
    -----------
    lista_columnas : list
        Lista con los nombres originales de las columnas
    mostrar_resumen : bool, default=True
        Si True, imprime un resumen de los cambios realizados
    
    Retorna:
    --------
    list
        Lista de nombres de columnas normalizados en formato snake_case
        
    Ejemplo:
    --------
    >>> normalizar_nombres_columnas(['EmployeeNumber', 'BusinessTravel'])
    ['employee_number', 'business_travel']
    """
    nombres_normalizados = []
    
    for nombre in lista_columnas:
        # Paso 1: Eliminar espacios al inicio y al final
        limpia = nombre.strip()
        
        # Paso 2: Eliminar caracteres especiales, conservando solo letras, números y guiones bajos
        limpia = re.sub(r'[^0-9a-zA-Z_]', '', limpia)
        
        # Paso 3: Insertar guión bajo entre minúscula/número y mayúscula
        # Ejemplo: 'BusinessTravel' -> 'Business_Travel'
        limpia = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', limpia)
        
        # Paso 4: Convertir todo a minúsculas
        nombres_normalizados.append(limpia.lower())

    # Mostrar resumen de cambios si se solicita
    if mostrar_resumen:
        print("Normalización de nombres de columnas finalizada.")
        print(f"Total columnas procesadas: {len(nombres_normalizados)}")
        print("Resumen de cambios:")
        for orig, nuevo in zip(lista_columnas, nombres_normalizados):
            print(f"'{orig}' -> '{nuevo}'")
    
    return nombres_normalizados

In [36]:
# Aplicar la normalización de nombres a todas las columnas del DataFrame
df.columns = normalizar_nombres_columnas(df.columns.tolist(), mostrar_resumen=True)

Normalización de nombres de columnas finalizada.
Total columnas procesadas: 35
Resumen de cambios:
'Age' -> 'age'
'Attrition' -> 'attrition'
'BusinessTravel' -> 'business_travel'
'DailyRate' -> 'daily_rate'
'Department' -> 'department'
'DistanceFromHome' -> 'distance_from_home'
'Education' -> 'education'
'EducationField' -> 'education_field'
'EmployeeCount' -> 'employee_count'
'EmployeeNumber' -> 'employee_number'
'EnvironmentSatisfaction' -> 'environment_satisfaction'
'Gender' -> 'gender'
'HourlyRate' -> 'hourly_rate'
'JobInvolvement' -> 'job_involvement'
'JobLevel' -> 'job_level'
'JobRole' -> 'job_role'
'JobSatisfaction' -> 'job_satisfaction'
'MaritalStatus' -> 'marital_status'
'MonthlyIncome' -> 'monthly_income'
'MonthlyRate' -> 'monthly_rate'
'NumCompaniesWorked' -> 'num_companies_worked'
'Over18' -> 'over18'
'OverTime' -> 'over_time'
'PercentSalaryHike' -> 'percent_salary_hike'
'PerformanceRating' -> 'performance_rating'
'RelationshipSatisfaction' -> 'relationship_satisfaction'
'S

---
## **2. ESTABLECER COLUMNA IDENTIFICADORA COMO ÍNDICE**

Usamos la columna de número de empleado como índice del DataFrame para:
- Facilitar el acceso a registros individuales
- Mejorar la eficiencia en búsquedas
- Dar estructura lógica al dataset

In [37]:
def usar_columna_como_indice(df, columna_original='employee_number', indice='id'):
    """
    Establece una columna del DataFrame como índice y la renombra.
    
    Esta función es útil para identificar de forma única cada registro
    y facilitar el acceso a los datos.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame a modificar
    columna_original : str, default='employee_number'
        Nombre de la columna que se usará como índice
    indice : str, default='id'
        Nuevo nombre para el índice (normalmente 'id')
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame con la columna establecida como índice y renombrada
        
    Raises:
    -------
    ValueError
        Si la columna especificada no existe en el DataFrame
    """
    # Crear copia para no modificar el DataFrame original
    df = df.copy()
    
    # Verificar que la columna existe antes de intentar usarla
    if columna_original not in df.columns:
        raise ValueError(f"La columna '{columna_original}' no existe en el DataFrame.")
    
    # Renombrar la columna al nombre corto deseado
    df.rename(columns={columna_original: indice}, inplace=True)
    
    # Establecer la columna renombrada como índice del DataFrame
    df.set_index(indice, inplace=True)
    
    print(f"Columna '{columna_original}' renombrada a '{indice}' y establecida como índice correctamente.")
    
    return df

In [38]:
# Establecer employee_number como índice y renombrarlo a 'id'
df = usar_columna_como_indice(df, columna_original='employee_number', indice='id')

Columna 'employee_number' renombrada a 'id' y establecida como índice correctamente.


---
## **3. ELIMINACIÓN DE FILAS DUPLICADAS**

Identificamos y eliminamos registros duplicados para:
- Evitar sesgos en el análisis
- Mantener la integridad de los datos
- Mejorar la calidad del dataset

In [39]:
def eliminar_filas_duplicadas(df, keep='first'):
    """
    Elimina filas duplicadas del DataFrame.
    
    Los duplicados son filas que tienen valores idénticos en todas las columnas.
    Esta función permite controlar qué ocurrencia del duplicado se conserva.

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame a procesar
    keep : {'first', 'last', False}, default='first'
        Determina qué duplicados conservar:
        - 'first'  : conserva la primera aparición de cada duplicado
        - 'last'   : conserva la última aparición de cada duplicado
        - False    : elimina todas las filas duplicadas (ninguna se conserva)

    Retorna:
    --------
    pd.DataFrame
        DataFrame sin filas duplicadas
    """
    # Crear copia para no modificar el DataFrame original
    df = df.copy()
    
    # Guardar el número de filas antes de la limpieza
    num_filas_antes_limpieza = df.shape[0]

    # Eliminar filas duplicadas según el criterio especificado
    df.drop_duplicates(keep=keep, inplace=True)

    # Calcular cuántas filas fueron eliminadas
    filas_eliminadas = num_filas_antes_limpieza - df.shape[0]

    # Mostrar resumen del proceso
    print(f"Filas antes de eliminar duplicados: {num_filas_antes_limpieza}")
    print(f"Filas eliminadas por duplicados: {filas_eliminadas}")
    print(f"Filas en el DataFrame tras el procesamiento: {df.shape[0]}")

    return df

In [40]:
# Eliminar duplicados conservando la primera aparición de cada registro
df = eliminar_filas_duplicadas(df, keep='first')

Filas antes de eliminar duplicados: 1474
Filas eliminadas por duplicados: 4
Filas en el DataFrame tras el procesamiento: 1470


---
## **4. ELIMINACIÓN DE COLUMNAS SIN APORTE ANALÍTICO**

Eliminamos columnas que no aportan información útil:
- **Columnas constantes**: todas las filas tienen el mismo valor
- **Columnas con alta cardinalidad**: casi todos los valores son únicos

Esto reduce dimensionalidad y mejora la eficiencia del análisis.

In [41]:
def eliminar_columnas_sin_aporte_analitico(df, umbral_cardinalidad=0.95):
    """
    Elimina columnas del DataFrame que no aportan valor analítico.
    
    Se consideran sin aporte analítico:
    - Columnas constantes: todos los valores son iguales (no aportan variabilidad)
    - Columnas con alta cardinalidad: la proporción de valores únicos es muy alta

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame a limpiar
    umbral_cardinalidad : float, default=0.95
        Proporción máxima de valores únicos permitida (entre 0 y 1)
        Si la cardinalidad supera este umbral, la columna se elimina
        Por defecto 0.95 significa que si más del 95% de valores son únicos, se elimina

    Retorna:
    --------
    pd.DataFrame
        DataFrame con las columnas problemáticas eliminadas
    """
    # Crear copia para no modificar el DataFrame original
    df = df.copy()
    
    # Lista para almacenar nombres de columnas a eliminar
    columnas_a_eliminar = []

    # Revisar cada columna del DataFrame
    for columna in df.columns:
        # Caso 1: Columna constante (todos los valores son iguales)
        # nunique() cuenta cuántos valores únicos hay
        if df[columna].nunique() == 1:
            columnas_a_eliminar.append(columna)
            
        # Caso 2: Columna con alta cardinalidad
        # Calculamos la proporción de valores únicos respecto al total de filas
        elif df[columna].nunique() / len(df) > umbral_cardinalidad:
            columnas_a_eliminar.append(columna)

    # Eliminar todas las columnas identificadas
    df.drop(columns=columnas_a_eliminar, inplace=True)

    # Mostrar resumen de la operación
    print(f"Columnas eliminadas por no aportar valor analítico: {columnas_a_eliminar}")
    print(f"Total columnas eliminadas: {len(columnas_a_eliminar)}")
    print(f"Columnas restantes en el DataFrame: {df.shape[1]}")

    return df

In [42]:
# Eliminar columnas constantes y con alta cardinalidad (>95% valores únicos)
df = eliminar_columnas_sin_aporte_analitico(df, umbral_cardinalidad=0.95)

Columnas eliminadas por no aportar valor analítico: ['employee_count', 'monthly_rate', 'over18', 'standard_hours']
Total columnas eliminadas: 4
Columnas restantes en el DataFrame: 30


---
## **5. NORMALIZACIÓN DE COLUMNAS DE TEXTO**

Estandarizamos los valores de texto para:
- Eliminar espacios innecesarios
- Unificar formato (Title Case)
- Corregir errores tipográficos
- Simplificar categorías

In [43]:
def normalizar_columnas_texto(df, mapeos_reemplazo=None, mostrar_resumen=True):
    """
    Normaliza columnas de texto o categóricas y aplica correcciones específicas.
    
    El proceso incluye:
    1. Eliminar espacios en blanco al inicio y al final de cada valor
    2. Convertir texto a Title Case (Primera Letra Mayúscula)
    3. Aplicar mapeos de reemplazo personalizados (correcciones, simplificaciones)

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame a procesar
    mapeos_reemplazo : dict, optional
        Diccionario de mapeos para corregir valores específicos
        Formato: {'nombre_columna': {'valor_antiguo': 'valor_nuevo', ...}, ...}
        Ejemplo: {'marital_status': {'Marreid': 'Married'}}
    mostrar_resumen : bool, default=True
        Si True, imprime resumen de columnas procesadas y categorías finales

    Retorna:
    --------
    pd.DataFrame
        DataFrame con columnas de texto normalizadas
    """
    # Crear copia para no modificar el DataFrame original
    df = df.copy()
    
    # Identificar columnas de texto/categóricas
    columnas_texto = df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Lista para rastrear columnas donde se aplicaron reemplazos
    columnas_con_reemplazo = []

    for col in columnas_texto:
        # Paso 1: Eliminar espacios al inicio/final y convertir a Title Case
        # .str.strip() elimina espacios en blanco
        # .str.title() convierte a formato "Primera Letra Mayúscula"
        df[col] = df[col].str.strip().str.title()

        # Paso 2: Aplicar mapeos de reemplazo específicos si existen
        if mapeos_reemplazo and col in mapeos_reemplazo:
            df[col] = df[col].replace(mapeos_reemplazo[col])
            columnas_con_reemplazo.append(col)

    # Mostrar resumen del proceso
    if mostrar_resumen:
        print("Normalización de columnas de texto finalizada.")
        print(f"Columnas procesadas: {columnas_texto}")

        if columnas_con_reemplazo:
            print(f"Se aplicaron mapeos de reemplazo en: {columnas_con_reemplazo}")
            for col in columnas_con_reemplazo:
                print(f"Categorías finales de '{col}': {df[col].unique().tolist()}")

    return df

In [44]:
# Definir mapeos para corregir errores tipográficos y simplificar valores
mapeos_reemplazo = {
    # Corregir error tipográfico en estado civil
    'marital_status': {'Marreid': 'Married'},
    
    # Simplificar categorías de frecuencia de viaje
    'business_travel': {
        'Travel_Rarely': 'Rarely',
        'Travel_Frequently': 'Frequently',
        'Non-Travel': 'Non'
    }
}

# Aplicar normalización de texto con los mapeos definidos
df = normalizar_columnas_texto(df, mapeos_reemplazo=mapeos_reemplazo)

Normalización de columnas de texto finalizada.
Columnas procesadas: ['attrition', 'business_travel', 'department', 'education_field', 'gender', 'job_role', 'marital_status', 'over_time']
Se aplicaron mapeos de reemplazo en: ['business_travel', 'marital_status']
Categorías finales de 'business_travel': ['Rarely', 'Frequently', 'Non', nan]
Categorías finales de 'marital_status': ['Single', 'Married', 'Divorced', nan]


---
## **6. CONVERSIÓN DE TIPOS DE DATOS**

Convertimos columnas a los tipos de datos apropiados para:
- Reducir consumo de memoria
- Permitir operaciones matemáticas correctas
- Manejar valores nulos de forma adecuada (Int64 vs int64)

In [45]:
def convertir_tipos_columnas(df, mapeo_tipos, mostrar_resumen=True):
    """
    Convierte columnas de un DataFrame a tipos de datos específicos.
    
    Esta función es útil para optimizar el uso de memoria y asegurar
    que cada columna tenga el tipo de dato correcto para su análisis.

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame a modificar
    mapeo_tipos : dict
        Diccionario con nombres de columnas como claves y tipos de datos como valores
        Tipos comunes: int, float, 'Int64', 'Float64', 'category', 'str'
        Nota: 'Int64' (mayúscula) permite valores nulos, 'int64' (minúscula) no
        Ejemplo: {'age': 'Int64', 'salary': float}
    mostrar_resumen : bool, default=True
        Si True, imprime resumen de conversiones y tabla de tipos finales

    Retorna:
    --------
    pd.DataFrame
        DataFrame con columnas convertidas a los tipos especificados
    """
    # Crear copia para no modificar el DataFrame original
    df = df.copy()
    
    # Lista para rastrear columnas convertidas exitosamente
    columnas_convertidas = []

    # Intentar convertir cada columna especificada
    for columna, tipo in mapeo_tipos.items():
        # Verificar que la columna existe en el DataFrame
        if columna in df.columns:
            try:
                # Intentar la conversión de tipo
                # errors='ignore' evita errores si la conversión falla en algunos valores
                df[columna] = df[columna].astype(tipo, errors='ignore')
                columnas_convertidas.append(columna)
            except Exception as e:
                # Si falla, informar del error pero continuar con otras columnas
                print(f"No se pudo convertir la columna '{columna}' a {tipo}: {e}")

    # Mostrar resumen del proceso
    if mostrar_resumen:
        print("Conversión de tipos finalizada.")
        if columnas_convertidas:
            print(f"Columnas convertidas: {columnas_convertidas}")
        else:
            print("No se convirtió ninguna columna.")

        # Mostrar tabla con todos los tipos de datos finales
        print("\nTipos de datos finales por columna:")
        tipos_finales = pd.DataFrame({
            "Columna": df.columns,
            "Tipo de dato": [df[col].dtype for col in df.columns]
        })
        display(tipos_finales)
            
    return df

In [46]:
# Definir el mapeo de tipos de datos para cada columna
mapeo_tipos = {
    "age": "Int64",                      # Int64 permite valores nulos (nullable integer)
    "daily_rate": float,                 # Tasa diaria como número decimal
    "hourly_rate": float,                # Tasa horaria como número decimal
    "training_times_last_year": "Int64", # Número de entrenamientos (puede tener nulos)
    "years_with_curr_manager": "Int64",  # Años con el manager actual (puede tener nulos)
}

# Aplicar la conversión de tipos
df = convertir_tipos_columnas(df, mapeo_tipos)

Conversión de tipos finalizada.
Columnas convertidas: ['age', 'daily_rate', 'hourly_rate', 'training_times_last_year', 'years_with_curr_manager']

Tipos de datos finales por columna:


,Columna,Tipo de dato
0,age,Int64
1,attrition,object
2,business_travel,object
3,daily_rate,float64
4,department,object
5,distance_from_home,int64
6,education,int64
7,education_field,object
8,environment_satisfaction,int64
9,gender,object


---
## **7. MAPEO DE VARIABLES ORDINALES**

Convertimos valores numéricos codificados en etiquetas semánticas para:
- Mejorar la interpretabilidad (1→"Becario" es más claro que 1)
- Facilitar la comunicación de resultados
- Mantener el orden lógico de las categorías

In [47]:
def mapear_columnas_ordinales(df, mapeos_columnas, mostrar_resumen=True):
    """
    Aplica mapeo semántico a columnas ordinales del DataFrame.
    
    Las variables ordinales son aquellas que tienen un orden natural
    (ej: 1=Bajo, 2=Medio, 3=Alto). Esta función las convierte de códigos
    numéricos a etiquetas textuales más comprensibles.

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame a modificar
    mapeos_columnas : dict
        Diccionario con nombres de columnas como claves y diccionarios de mapeo como valores
        Formato: {'columna': {valor_numerico: 'etiqueta_texto', ...}, ...}
        Ejemplo:
        {
            "education": {1: "Sin estudios", 2: "Educación básica", ...},
            "job_level": {1: "Becario", 2: "Junior", ...}
        }
    mostrar_resumen : bool, default=True
        Si True, imprime resumen de columnas mapeadas y sus valores únicos

    Retorna:
    --------
    pd.DataFrame
        DataFrame con columnas ordinales mapeadas a etiquetas textuales
    """
    # Crear copia para no modificar el DataFrame original
    df = df.copy()
    
    # Lista para rastrear columnas mapeadas
    columnas_mapeadas = []

    # Aplicar cada mapeo especificado
    for columna, mapa in mapeos_columnas.items():
        # Verificar que la columna existe
        if columna in df.columns:
            # .map() reemplaza cada valor según el diccionario proporcionado
            # Ej: si mapa = {1: "Bajo", 2: "Alto"}, entonces 1 se convierte en "Bajo"
            df[columna] = df[columna].map(mapa)
            columnas_mapeadas.append(columna)

    # Mostrar resumen del proceso
    if mostrar_resumen:
        print("🔹 Mapeo de columnas ordinales finalizado.")
        print(f"Columnas mapeadas: {columnas_mapeadas}")
        # Mostrar valores únicos de cada columna mapeada para verificación
        for col in columnas_mapeadas:
            print(f"{col}: {df[col].unique()}")

    return df

In [48]:
# ============================================================================
# DEFINICIÓN DE MAPEOS PARA VARIABLES ORDINALES
# ============================================================================

# Mapeo para variables de satisfacción (escala 1-4)
satisfaction_map = {
    1: "Nada satisfecho",
    2: "Insatisfecho",
    3: "Satisfecho",
    4: "Muy satisfecho",
}

# Mapeo para nivel educativo (escala 1-5)
education_map = {
    1: "Sin estudios",
    2: "Educación básica",
    3: "FP/Bachiller",
    4: "Estudios universitarios",
    5: "Postgrado",
}

# Mapeo para nivel de puesto (escala 1-5)
job_level_map = {
    1: "Becario",
    2: "Junior",
    3: "Senior",
    4: "Manager",
    5: "Director",
}

# Lista de columnas que usan la escala de satisfacción
satisfaction_cols = [
    "env_satisfaction",      # Satisfacción con el ambiente
    "job_involvement",       # Involucramiento en el trabajo
    "job_satisfaction",      # Satisfacción laboral
    "performance_score",     # Autoevaluación de desempeño
    "rel_satisfaction",      # Satisfacción con relaciones
    "work_life_balance",     # Balance vida-trabajo
]

# Construir diccionario completo de mapeos
# Todas las columnas de satisfacción usan el mismo mapeo
mapeos_ordinales = {col: satisfaction_map for col in satisfaction_cols}

# Añadir mapeos específicos para educación y nivel de puesto
mapeos_ordinales["education"] = education_map
mapeos_ordinales["job_level"] = job_level_map

# Aplicar todos los mapeos al DataFrame
df = mapear_columnas_ordinales(df, mapeos_ordinales)

🔹 Mapeo de columnas ordinales finalizado.
Columnas mapeadas: ['job_involvement', 'job_satisfaction', 'work_life_balance', 'education', 'job_level']
job_involvement: ['Satisfecho' 'Insatisfecho' 'Muy satisfecho' 'Nada satisfecho']
job_satisfaction: ['Muy satisfecho' 'Insatisfecho' 'Satisfecho' 'Nada satisfecho' nan]
work_life_balance: ['Nada satisfecho' 'Satisfecho' 'Insatisfecho' 'Muy satisfecho']
education: ['Educación básica' 'Sin estudios' 'Estudios universitarios'
 'FP/Bachiller' 'Postgrado']
job_level: ['Junior' 'Becario' 'Senior' 'Manager' 'Director']


---
## **8. IMPUTACIÓN DE VALORES NULOS EN VARIABLES CATEGÓRICAS**

Rellenamos valores faltantes en variables categóricas siguiendo reglas basadas en:
- **Porcentaje de nulos**: alto (>20%), medio (5-20%), bajo (<5%)
- **Dominancia de la moda**: qué tan frecuente es la categoría más común
- **Ventaja de la moda**: diferencia con la segunda categoría más frecuente

**Estrategias:**
- Muchos nulos (>20%) → `Unknown`
- Pocos nulos con moda dominante → Moda
- Resto de casos → `Unknown`

In [49]:
def imputar_categoricas(df, columnas, umbral_nulos_alto=0.20, umbral_nulos_bajo=0.05,
                        umbral_moda_bajo=0.50, umbral_moda_medio=0.60, umbral_ventaja=0.20,
                        etiqueta_unknown="Unknown"):
    """
    Imputa valores nulos en variables categóricas siguiendo reglas justificables estadísticamente.
    
    LÓGICA DE IMPUTACIÓN:
    
    1) % de nulos ALTO (> 20%):
       → Imputar con 'Unknown'
       → Justificación: Con muchos nulos, imputar por moda inventa demasiada información
    
    2) % de nulos BAJO (≤ 5%) o MEDIO (5-20%):
       → Imputar con MODA solo si hay una categoría verdaderamente dominante
       → Condiciones para usar moda:
          a) La moda debe superar un umbral mínimo:
             - Nulos bajos: moda ≥ 50% del total
             - Nulos medios: moda ≥ 60% del total (más exigente)
          b) La moda debe tener ventaja sobre la 2ª categoría:
             - (% moda - % segunda) ≥ 20 puntos porcentuales
       → Si NO se cumplen estas condiciones → 'Unknown'

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame a procesar (se modifica)
    columnas : list[str]
        Lista de columnas categóricas donde imputar nulos
    umbral_nulos_alto : float, default=0.20
        Umbral para considerar % de nulos como "alto"
    umbral_nulos_bajo : float, default=0.05
        Umbral para considerar % de nulos como "bajo"
    umbral_moda_bajo : float, default=0.50
        % mínimo que debe tener la moda cuando los nulos son bajos
    umbral_moda_medio : float, default=0.60
        % mínimo que debe tener la moda cuando los nulos son medios
    umbral_ventaja : float, default=0.20
        Ventaja mínima (en puntos porcentuales) entre moda y 2ª categoría
    etiqueta_unknown : str, default="Unknown"
        Etiqueta para imputar cuando no se usa la moda

    Retorna:
    --------
    pd.DataFrame
        DataFrame con nulos imputados en las columnas especificadas
    """

    total = len(df)

    for col in columnas:
        print(f"\n📌 Analizando columna: {col}")
        
        # Validar que la columna existe
        if col not in df.columns:
            print(f"❌ La columna {col} no existe en el DataFrame. Se omite.")
            continue

        # Contar nulos y calcular porcentaje
        nulos = df[col].isnull().sum()
        porcentaje_nulos = nulos / total if total > 0 else 0

        print(f"   → Nulos: {nulos} de {total} ({porcentaje_nulos:.2%})")

        # ================================================================
        # REGLA 1: Muchos nulos → Unknown directamente
        # ================================================================
        if porcentaje_nulos > umbral_nulos_alto:
            print(
                f"   🔴 Porcentaje de nulos > {umbral_nulos_alto:.0%} "
                f"→ se crea la categoría '{etiqueta_unknown}'"
            )
            df[col] = df[col].fillna(etiqueta_unknown)
            continue
        
        # Calcular frecuencias de categorías (sin contar nulos)
        valores = df[col].value_counts(dropna=True)

        # Si no hay valores no nulos, imputar con Unknown
        if len(valores) == 0:
            print(
                f"   🔴 No hay valores no nulos para decidir moda "
                f"→ se crea la categoría '{etiqueta_unknown}'"
            )
            df[col] = df[col].fillna(etiqueta_unknown)
            continue

        # Obtener moda (categoría más frecuente)
        primero = valores.iloc[0]  # Frecuencia absoluta de la moda
        pct_primero = primero / total  # Proporción sobre el total de filas

        # Obtener segunda categoría más frecuente (si existe)
        if len(valores) > 1:
            segundo = valores.iloc[1]
            pct_segundo = segundo / total
        else:
            pct_segundo = 0.0

        # Calcular ventaja de la moda sobre la 2ª categoría
        ventaja = pct_primero - pct_segundo

        # Mostrar información de las categorías principales
        print(f"   → Moda: {valores.index[0]} ({pct_primero:.2%})")
        print(f"   → 2ª categoría: {valores.index[1] if len(valores) > 1 else 'No existe'} ({pct_segundo:.2%})")
        print(f"   → Ventaja de la moda: {ventaja:.2%}")

        # ================================================================
        # REGLA 2: Determinar umbral de moda según % de nulos
        # ================================================================
        if porcentaje_nulos <= umbral_nulos_bajo:
            umbral_moda = umbral_moda_bajo
            print(
                f"   → Nulos bajos (≤ {umbral_nulos_bajo:.0%}), "
                f"umbral de moda requerido: {umbral_moda:.0%}"
            )
        else:  # Nulos medios (entre bajo y alto)
            umbral_moda = umbral_moda_medio
            print(
                f"   → Nulos medios (> {umbral_nulos_bajo:.0%} y ≤ {umbral_nulos_alto:.0%}), "
                f"umbral de moda requerido: {umbral_moda:.0%}"
            )
        
        # ================================================================
        # REGLA 3: Imputar por moda solo si es dominante Y tiene ventaja
        # ================================================================
        if (pct_primero >= umbral_moda) and (ventaja >= umbral_ventaja):
            moda = df[col].mode(dropna=True)[0]
            print(
                f"   🟢 La moda es dominante y con ventaja suficiente "
                f"(≥ {umbral_ventaja:.0%}) → se imputan nulos con '{moda}'"
            )
            df[col] = df[col].fillna(moda)
        else:
            print(
                f"   🟡 La moda NO es lo suficientemente dominante "
                f"→ se crea la categoría '{etiqueta_unknown}'"
            )
            df[col] = df[col].fillna(etiqueta_unknown)

    return df

In [50]:
# ============================================================================
# PREPARACIÓN Y APLICACIÓN DE IMPUTACIÓN CATEGÓRICA
# ============================================================================

# Paso 1: Identificar columnas categóricas (object o category)
cols_cat = df.select_dtypes(include=["object", "category"]).columns.tolist()

# Paso 2: Excluir la variable objetivo 'attrition' de la imputación
# (no queremos imputar la variable que es objeto de análisis)
cols_cat = [c for c in cols_cat if c.lower() != "attrition"]

# Paso 3: Normalizar espacios en blanco vacíos a NaN
# Algunos datasets tienen celdas con solo espacios que pandas no detecta como nulos
# El regex r'^\s*$' busca cadenas que solo contengan espacios (o estén vacías)
df[cols_cat] = df[cols_cat].replace(r'^\s*$', pd.NA, regex=True)

# Paso 4: Aplicar imputación con las reglas definidas
df = imputar_categoricas(df, cols_cat)


📌 Analizando columna: business_travel
   → Nulos: 117 de 1470 (7.96%)
   → Moda: Rarely (64.69%)
   → 2ª categoría: Frequently (17.89%)
   → Ventaja de la moda: 46.80%
   → Nulos medios (> 5% y ≤ 20%), umbral de moda requerido: 60%
   🟢 La moda es dominante y con ventaja suficiente (≥ 20%) → se imputan nulos con 'Rarely'

📌 Analizando columna: department
   → Nulos: 29 de 1470 (1.97%)
   → Moda: Research & Development (63.88%)
   → 2ª categoría: Sales (29.86%)
   → Ventaja de la moda: 34.01%
   → Nulos bajos (≤ 5%), umbral de moda requerido: 50%
   🟢 La moda es dominante y con ventaja suficiente (≥ 20%) → se imputan nulos con 'Research & Development'

📌 Analizando columna: education
   → Nulos: 0 de 1470 (0.00%)
   → Moda: FP/Bachiller (38.91%)
   → 2ª categoría: Estudios universitarios (27.07%)
   → Ventaja de la moda: 11.84%
   → Nulos bajos (≤ 5%), umbral de moda requerido: 50%
   🟡 La moda NO es lo suficientemente dominante → se crea la categoría 'Unknown'

📌 Analizando columna: e

---
## **9. IMPUTACIÓN DE VALORES NULOS EN VARIABLES NUMÉRICAS**

Rellenamos valores faltantes en variables numéricas usando estrategias robustas:

**Estrategias según % de nulos:**
- **Nulos bajos (≤5%)**: Mediana (robusta a outliers)
- **Nulos medios (5-20%)**: KNN con escalado (usa información de registros similares)
- **Nulos altos (>20%)**: Mediana + indicador de missingness

**¿Por qué escalado con KNN?**
KNN calcula distancias entre filas. Si una variable está en escala 0-10 y otra en 0-10,000,
la segunda dominaría el cálculo. El escalado (StandardScaler) pone todas las variables
en la misma escala antes de calcular similitudes.

In [51]:
def imputar_numericas(df, columnas, umbral_nulos_bajo=0.05, umbral_nulos_alto=0.20,
                      n_neighbors=5, crear_indicador_missing=True, usar_knn_en_alto=False):
    """
    Imputa valores nulos en variables numéricas con estrategias robustas.
    
    LÓGICA DE IMPUTACIÓN:
    
    1) % de nulos BAJO (≤ 5%):
       → Imputar con MEDIANA (SimpleImputer)
       → Justificación: Con pocos nulos, la mediana es estable y robusta a outliers
    
    2) % de nulos MODERADO (5-20%):
       → Imputar con KNN (KNNImputer) usando escalado
       → Justificación: KNN aprovecha información de registros similares
       → Requiere escalado porque KNN usa distancias entre filas
    
    3) % de nulos ALTO (> 20%):
       → Crear variable indicadora de missingness (col_missing: 0/1)
       → Imputar con MEDIANA (por defecto) o KNN (si usar_knn_en_alto=True)
       → Justificación: La presencia de nulo puede ser informativa
    
    IMPORTANTE SOBRE KNN Y ESCALADO:
    KNN calcula similitud entre filas usando distancias. Si las columnas están en
    escalas diferentes (ej: una en 0-10, otra en 0-10000), la de mayor rango
    dominaría el cálculo. Por eso:
      1. Escalamos (StandardScaler): (x - media) / desviación
      2. Aplicamos KNN en el espacio escalado
      3. Desescalamos para volver a unidades originales

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame a procesar (se modifica)
    columnas : list[str]
        Lista de columnas numéricas donde imputar nulos
    umbral_nulos_bajo : float, default=0.05
        Umbral para % de nulos "bajo"
    umbral_nulos_alto : float, default=0.20
        Umbral para % de nulos "alto"
    n_neighbors : int, default=5
        Número de vecinos (k) para KNNImputer
    crear_indicador_missing : bool, default=True
        Si True, crea columna col_missing cuando % nulos es alto
    usar_knn_en_alto : bool, default=False
        Si True, usa KNN también cuando % nulos es alto (más costoso)

    Retorna:
    --------
    pd.DataFrame
        DataFrame con nulos imputados en las columnas especificadas
    """
    
    #Guardar los tipos originales para restaurar a int de nuevo tras la imputación
    dtypes_originales = df.dtypes.copy()
    total = len(df)

    # Filtrar solo columnas que existen 
    columnas_validas = df[columnas].select_dtypes(include=[np.number]).columns.tolist()
    columnas_no_encontradas = [c for c in columnas if c not in df.columns]
    
    for c in columnas_no_encontradas:
        print(f"{c} no existe en el DataFrame. Se omite.")

    # Verificar que son numéricas
    cols_num_df = df.select_dtypes(include=[np.number]).columns.tolist()
    columnas_validas = [c for c in columnas_validas if c in cols_num_df]

    if len(columnas_validas) == 0:
        print("No hay columnas numéricas válidas para imputar.")
        return df

    # Preparar imputador de mediana (se reutiliza)
    imputer_mediana = SimpleImputer(strategy="median")

    # Para KNN usamos todas las numéricas como contexto (mejora la similitud)
    cols_num_contexto = cols_num_df

    for col in columnas_validas:
        print(f"\n📌 Analizando columna numérica: {col}")

        nulos = df[col].isnull().sum()
        porcentaje_nulos = nulos / total if total > 0 else 0
        print(f"   → Nulos: {nulos} de {total} ({porcentaje_nulos:.2%})")

        # ================================================================
        # CASO 1: Nulos bajos → MEDIANA
        # ================================================================
        if porcentaje_nulos <= umbral_nulos_bajo:
            print(
                f"   🟢 Nulos bajos (≤ {umbral_nulos_bajo:.0%}) "
                f"→ imputación con MEDIANA (SimpleImputer)"
            )
            df[[col]] = imputer_mediana.fit_transform(df[[col]])
            continue

        # ================================================================
        # CASO 2: Nulos moderados → KNN con ESCALADO
        # ================================================================
        if porcentaje_nulos <= umbral_nulos_alto:
            print(
                f"   🟡 Nulos moderados (> {umbral_nulos_bajo:.0%} y ≤ {umbral_nulos_alto:.0%}) "
                f"→ imputación con KNN (k={n_neighbors}) + ESCALADO"
            )

            # Paso 1: Extraer bloque numérico completo (para contexto de similitud)
            X = df[cols_num_contexto].copy()

            # Paso 2: ESCALAR - poner todas las columnas en escala comparable
            # StandardScaler: z = (x - media) / desviación_estándar
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            # Paso 3: Imputar usando KNN en el espacio escalado
            knn = KNNImputer(n_neighbors=n_neighbors)
            X_imputed_scaled = knn.fit_transform(X_scaled)

            # Paso 4: DESESCALAR - volver a las unidades originales
            X_imputed = scaler.inverse_transform(X_imputed_scaled)

            # Paso 5: Convertir de vuelta a DataFrame
            X_imputed = pd.DataFrame(X_imputed, columns=cols_num_contexto, index=df.index)

            # Paso 6: Guardar solo la columna objetivo (no tocamos otras numéricas)
            df[col] = X_imputed[col]

            print(f"   ✅ {col} imputada con KNN + escalado (solo se asigna esta columna).")
            continue

        # ================================================================
        # CASO 3: Nulos altos → INDICADOR + MEDIANA/KNN
        # ================================================================
        print(
            f"   🔴 Nulos altos (> {umbral_nulos_alto:.0%}) "
            f"→ se considera missingness + imputación robusta"
        )

        # Crear variable indicadora de nulo (1 si era nulo, 0 si no)
        if crear_indicador_missing:
            indicador = f"{col}_missing"
            df[indicador] = df[col].isnull().astype(int)
            print(f"   → Se crea indicador de missingness: {indicador} (1=nulo, 0=no nulo)")

        # Decidir método de imputación
        if usar_knn_en_alto:
            print(
                f"   🟠 usar_knn_en_alto=True "
                f"→ imputación con KNN (k={n_neighbors}) + ESCALADO"
            )

            X = df[cols_num_contexto].copy()
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            knn = KNNImputer(n_neighbors=n_neighbors)
            X_imputed_scaled = knn.fit_transform(X_scaled)

            X_imputed = scaler.inverse_transform(X_imputed_scaled)
            X_imputed = pd.DataFrame(X_imputed, columns=cols_num_contexto, index=df.index)

            df[col] = X_imputed[col]
            print(f"   ✅ {col} imputada con KNN + escalado (solo se asigna esta columna).")

        else:
            print("   🟠 Se imputa con MEDIANA (SimpleImputer) por estabilidad.")
            df[[col]] = imputer_mediana.fit_transform(df[[col]])


    # Restaurar tipos de datos orginales (int + redondear + convertir)
    for col in columnas_validas:
        if col in dtypes_originales:
            # Si originalmente era entero → restaurar
            if pd.api.types.is_integer_dtype(dtypes_originales[col]):
                print(f"   🔄 Restaurando tipo entero en: {col}")
                df[col] = (
                    df[col]
                    .round()          # eliminar decimales de KNN
                    .astype("Int64")  # volver a int
                )
    return df

In [52]:
# ============================================================================
# PREPARACIÓN Y APLICACIÓN DE IMPUTACIÓN NUMÉRICA
# ============================================================================

# Paso 1: Identificar columnas numéricas (int o float)
cols_num = df.select_dtypes(include=[np.number]).columns[df.select_dtypes(include=[np.number]).isnull().any()].tolist()

# Paso 2: Definir las columnas específicas que queremos procesar
# (Podemos usar todas las numéricas detectadas o una lista personalizada)
# cols_a_imputar_num = ['age', 'monthly_income', 'total_working_years', ...] 
cols_a_imputar_num = cols_num 

# Paso 3: Aplicar imputación con lógica de escenarios (Mediana / KNN / Indicador)
df = imputar_numericas(
    df, 
    columnas=cols_a_imputar_num,
    umbral_nulos_bajo=0.05, 
    umbral_nulos_alto=0.20,
    n_neighbors=5,
    crear_indicador_missing=True,
    usar_knn_en_alto=False
)

# Paso 4: Verificación final de la limpieza
# Comprobamos que no queden nulos en las columnas procesadas
nulos_restantes = df[cols_a_imputar_num].isnull().sum().sum()
print(f"✅ Proceso finalizado. Nulos numéricos restantes: {nulos_restantes}")


📌 Analizando columna numérica: age
   → Nulos: 73 de 1470 (4.97%)
   🟢 Nulos bajos (≤ 5%) → imputación con MEDIANA (SimpleImputer)

📌 Analizando columna numérica: monthly_income
   → Nulos: 14 de 1470 (0.95%)
   🟢 Nulos bajos (≤ 5%) → imputación con MEDIANA (SimpleImputer)

📌 Analizando columna numérica: training_times_last_year
   → Nulos: 88 de 1470 (5.99%)
   🟡 Nulos moderados (> 5% y ≤ 20%) → imputación con KNN (k=5) + ESCALADO
   ✅ training_times_last_year imputada con KNN + escalado (solo se asigna esta columna).

📌 Analizando columna numérica: years_with_curr_manager
   → Nulos: 147 de 1470 (10.00%)
   🟡 Nulos moderados (> 5% y ≤ 20%) → imputación con KNN (k=5) + ESCALADO
   ✅ years_with_curr_manager imputada con KNN + escalado (solo se asigna esta columna).
   🔄 Restaurando tipo entero en: age
   🔄 Restaurando tipo entero en: training_times_last_year
   🔄 Restaurando tipo entero en: years_with_curr_manager
✅ Proceso finalizado. Nulos numéricos restantes: 0


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1470 entries, 1 to 2068
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   age                         1470 non-null   Int64  
 1   attrition                   1470 non-null   object 
 2   business_travel             1470 non-null   object 
 3   daily_rate                  1470 non-null   float64
 4   department                  1470 non-null   object 
 5   distance_from_home          1470 non-null   int64  
 6   education                   1470 non-null   object 
 7   education_field             1470 non-null   object 
 8   environment_satisfaction    1470 non-null   int64  
 9   gender                      1470 non-null   object 
 10  hourly_rate                 1470 non-null   float64
 11  job_involvement             1470 non-null   object 
 12  job_level                   1470 non-null   object 
 13  job_role                    1470 non-n

---
## **10. FUNCIÓN DE LIMPIEZA GENERAL**

Esta función integra todos los pasos anteriores en un solo proceso automatizado.
Es útil cuando quieres aplicar todo el flujo de limpieza de una sola vez.

In [54]:
def limpieza_general(
    df,
    id_columna='employee_number',
    mapeo_tipos=None,
    mapeos_texto=None,
    mapeos_ordinales=None,
    columnas_categoricas_nulos=None,
    columnas_numericas_nulos=None,
    mostrar_resumen=True
):
    """
    Ejecuta el proceso completo de limpieza de datos en un DataFrame.
    
    Esta función orquesta todos los pasos de limpieza en el orden correcto:
    1. Normalización de nombres de columnas (snake_case)
    2. Establecer columna ID como índice
    3. Eliminación de filas duplicadas
    4. Eliminación de columnas sin aporte analítico
    5. Conversión de tipos de datos
    6. Normalización de columnas de texto
    7. Mapeo de columnas ordinales
    8. Imputación de nulos en categóricas
    9. Imputación de nulos en numéricas

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame original a limpiar
    id_columna : str, default='employee_number'
        Nombre de la columna que se usará como índice
    mapeo_tipos : dict, optional
        Diccionario de conversión de tipos {'columna': tipo}
    mapeos_texto : dict, optional
        Diccionario de reemplazos en texto {'columna': {'viejo': 'nuevo'}}
    mapeos_ordinales : dict, optional
        Diccionario de mapeos ordinales {'columna': {1: 'etiqueta'}}
    columnas_categoricas_nulos : list, optional
        Lista de columnas categóricas donde imputar nulos
    columnas_numericas_nulos : list, optional
        Lista de columnas numéricas donde imputar nulos
    mostrar_resumen : bool, default=True
        Si True, imprime resumen de cada paso

    Retorna:
    --------
    pd.DataFrame
        DataFrame limpio y listo para análisis
    """

    # Crear copia para no modificar el DataFrame original
    df = df.copy()

    # PASO 1: Normalizar nombres de columnas
    df.columns = normalizar_nombres_columnas(df.columns.tolist(), mostrar_resumen=mostrar_resumen)

    # PASO 2: Establecer ID como índice
    df = usar_columna_como_indice(df, columna_original=id_columna, indice='id')

    # PASO 3: Eliminar duplicados
    df = eliminar_filas_duplicadas(df)

    # PASO 4: Eliminar columnas sin aporte analítico
    df = eliminar_columnas_sin_aporte_analitico(df)

    # PASO 5: Conversión de tipos (si se proporcionó mapeo)
    if mapeo_tipos:
        df = convertir_tipos_columnas(df, mapeo_tipos, mostrar_resumen=mostrar_resumen)

    # PASO 6: Normalización de texto (si se proporcionaron mapeos)
    if mapeos_texto:
        df = normalizar_columnas_texto(df, mapeos_reemplazo=mapeos_texto, mostrar_resumen=mostrar_resumen)

    # PASO 7: Mapeo de ordinales (si se proporcionaron mapeos)
    if mapeos_ordinales:
        df = mapear_columnas_ordinales(df, mapeos_ordinales, mostrar_resumen=mostrar_resumen)

    # PASO 8: Imputación de categóricas (si se especificaron columnas)
    if columnas_categoricas_nulos:
        df = imputar_categoricas(df, columnas_categoricas_nulos)

    # PASO 9: Imputación de numéricas (si se especificaron columnas)
    if columnas_numericas_nulos:
        df = imputar_numericas(df, columnas_numericas_nulos)

    if mostrar_resumen:
        print("\n🟢 Limpieza general completada.")

    return df

---
## **11. EXPORTACIÓN DE DATOS LIMPIOS**

Guardamos el DataFrame procesado para su uso en análisis posteriores.

In [55]:
# Guardar el DataFrame limpio en la carpeta de datos procesados
# Este archivo será la fuente para análisis exploratorio y modelado
df.to_csv('../data/processed/hr_processed.csv')

print("\n✅ Datos limpios exportados exitosamente a '../data/processed/hr_processed.csv'")
print(f"📊 Dimensiones finales del dataset: {df.shape[0]} filas x {df.shape[1]} columnas")


✅ Datos limpios exportados exitosamente a '../data/processed/hr_processed.csv'
📊 Dimensiones finales del dataset: 1470 filas x 30 columnas
